# Audio Alignment for Harmonix Set

This notebook tries to align purchased audio with original audio from Harmonix. 

More specifically, for each pair of audio files:
- Load both audio files
- Use DTW to find the correct start and end points of alignment
- Produce the new aligned mp3s from the purchased audio

In [5]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

import librosa
from librosa import display

ORIG_MP3_PATH = "/home/uri/Projects/"
PURC_MP3_PATH = "/home/uri/Dropbox/drop/HarmonixMP3_YouTube/"
SR = 22050

%matplotlib inline

In [ ]:
# Load audio
_1, fs = librosa.load('audio/sir_duke_slow.mp3')
plt.figure(figsize=(16, 4))
librosa.display.waveplot(x_1, sr=fs)
plt.title('Slower Version $X_1$')
plt.tight_layout()